In [ ]:
# Importing the necessary packages as well as my own functions that will be used in this project.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from preprocessing import *

In [ ]:
# Read in the data, concat the df's if using more than one file. Also we merge it with a csv with player
# names and rename 'event_id' column to be compatible with the functions being used.
# cumulative_match_mins accounts for the stoppage time in games and insert_ball_carries inserts carries
# into the data.
# We then select a trimmed down df that we will use.

df7 = pd.read_csv("../whoscored data/pl_all_match_events.23.24.csv")
df1 = pd.read_csv("../whoscored data/pl_all_match_events.17.18.csv")
df2 = pd.read_csv("../whoscored data/pl_all_match_events.18.19.csv")
df3 = pd.read_csv("../whoscored data/pl_all_match_events.19.20.csv")
df4 = pd.read_csv("../whoscored data/pl_all_match_events.20.21.csv")
df5 = pd.read_csv("../whoscored data/pl_all_match_events.21.22.csv")
df6 = pd.read_csv("../whoscored data/pl_all_match_events.22.23.csv")

df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)
df = df.rename(columns={'event_id': 'eventId'})

merged_df = cumulative_match_mins(df)
carry_df = insert_ball_carries(merged_df)

df = carry_df.copy()
df = df[['match_id', 'eventId', 'expandedMinute', 'second', 'outcomeType', 'x', 'y', 'endX', 'endY', 'teamId', 'eventType', 'isShot']]

df_with_possessions = get_possession_chains(carry_df, chain_check=3, suc_evts_in_chain=2)